In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql import functions as fn, Row
spark = SparkSession.builder.appName('project_loan').getOrCreate()
sc = spark.sparkContext

In [2]:
df = spark.read.format("csv").\
    option("header", "true").\
    option("inferSchema", "true").\
     load("application_train.csv")

In [3]:
vars1=df.schema.names

In [4]:
len(vars1)

122

In [5]:
clean_df = df.dropna()

In [6]:
clean_df.count()

8602

In [7]:
df.limit(10).toPandas()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
6,100009,0,Cash loans,F,Y,Y,1,171000.0,1560726.0,41301.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0
7,100010,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,100011,0,Cash loans,F,N,Y,0,112500.0,1019610.0,33826.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
9,100012,0,Revolving loans,M,N,Y,0,135000.0,405000.0,20250.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.groupBy('TARGET').count().show()

+------+------+
|TARGET| count|
+------+------+
|     1| 24825|
|     0|282686|
+------+------+



In [9]:
df.groupBy('NAME_CONTRACT_TYPE').count().show()

+------------------+------+
|NAME_CONTRACT_TYPE| count|
+------------------+------+
|   Revolving loans| 29279|
|        Cash loans|278232|
+------------------+------+



In [10]:
df.groupBy('NAME_TYPE_SUITE').count().show()

+---------------+------+
|NAME_TYPE_SUITE| count|
+---------------+------+
|Spouse, partner| 11370|
|        Other_A|   866|
|           null|  1292|
|         Family| 40149|
|Group of people|   271|
|        Other_B|  1770|
|  Unaccompanied|248526|
|       Children|  3267|
+---------------+------+



In [11]:
df_clean = df.filter(df.NAME_TYPE_SUITE.isNotNull())

In [12]:
df_clean.groupBy('NAME_TYPE_SUITE').count().show()

+---------------+------+
|NAME_TYPE_SUITE| count|
+---------------+------+
|Spouse, partner| 11370|
|        Other_A|   866|
|         Family| 40149|
|Group of people|   271|
|        Other_B|  1770|
|  Unaccompanied|248526|
|       Children|  3267|
+---------------+------+



In [13]:
from pyspark.sql import functions as F

def quick_dummies(data, col_name, join_col):
    # select columns
    test = data.select(join_col, col_name)
    # map out all categories in column
    categories = test.select(col_name).distinct().rdd.flatMap(lambda x: x).collect()
    # creatre expression to append 1 for column that corresponds with dummy
    exprs = [F.when(F.col(col_name) == category, 1).otherwise(0).alias(category)
         for category in categories]
    # apply expression
    dummy=test.select(join_col, *exprs)
    
    return(dummy)

In [14]:
suite_dummy=quick_dummies(df_clean, 'NAME_TYPE_SUITE', 'SK_ID_CURR')

In [15]:
df_clean.groupBy('NAME_CONTRACT_TYPE').count().show()

+------------------+------+
|NAME_CONTRACT_TYPE| count|
+------------------+------+
|   Revolving loans| 28994|
|        Cash loans|277225|
+------------------+------+



In [16]:
df_clean.count()

306219

In [17]:
df_clean.groupby('NAME_INCOME_TYPE').count().show()

+--------------------+------+
|    NAME_INCOME_TYPE| count|
+--------------------+------+
|             Student|    17|
|Commercial associate| 71257|
|         Businessman|    10|
|     Maternity leave|     5|
|       State servant| 21612|
|             Working|158132|
|           Pensioner| 55164|
|          Unemployed|    22|
+--------------------+------+



In [18]:
income_dummy=quick_dummies(df_clean, 'NAME_INCOME_TYPE', 'SK_ID_CURR')

In [19]:
df_clean.groupby('NAME_EDUCATION_TYPE').count().show(30,False)

+-----------------------------+------+
|NAME_EDUCATION_TYPE          |count |
+-----------------------------+------+
|Academic degree              |163   |
|Incomplete higher            |10242 |
|Secondary / secondary special|217586|
|Lower secondary              |3810  |
|Higher education             |74418 |
+-----------------------------+------+



In [20]:
edu_dummy=quick_dummies(df_clean, 'NAME_EDUCATION_TYPE', 'SK_ID_CURR')

In [21]:
df_clean.groupby('NAME_FAMILY_STATUS').count().show()

+--------------------+------+
|  NAME_FAMILY_STATUS| count|
+--------------------+------+
|           Separated| 19695|
|             Married|195648|
|Single / not married| 45200|
|               Widow| 16015|
|      Civil marriage| 29661|
+--------------------+------+



In [22]:
fam_dummy=quick_dummies(df_clean, 'NAME_FAMILY_STATUS', 'SK_ID_CURR')

In [23]:
df_clean.groupby('NAME_HOUSING_TYPE').count().show()

+-------------------+------+
|  NAME_HOUSING_TYPE| count|
+-------------------+------+
|  House / apartment|271719|
|Municipal apartment| 11147|
|    Co-op apartment|  1113|
|   Rented apartment|  4863|
|   Office apartment|  2603|
|       With parents| 14774|
+-------------------+------+



In [24]:
house_dummy=quick_dummies(df_clean, 'NAME_HOUSING_TYPE', 'SK_ID_CURR')

In [25]:
df_clean.groupby('OCCUPATION_TYPE').count().show(30,False)

+---------------------+-----+
|OCCUPATION_TYPE      |count|
+---------------------+-----+
|null                 |96006|
|Managers             |21276|
|HR staff             |559  |
|Medicine staff       |8505 |
|Accountants          |9769 |
|Laborers             |54972|
|Cleaning staff       |4641 |
|Private service staff|2637 |
|Drivers              |18521|
|Sales staff          |31958|
|Realty agents        |748  |
|IT staff             |522  |
|Security staff       |6695 |
|Secretaries          |1301 |
|Low-skill Laborers   |2085 |
|Core staff           |27427|
|Cooking staff        |5930 |
|High skill tech staff|11327|
|Waiters/barmen staff |1340 |
+---------------------+-----+



In [26]:
df_clean = df_clean.filter(df_clean.OCCUPATION_TYPE.isNotNull())

In [27]:
occ_dummy=quick_dummies(df_clean, 'OCCUPATION_TYPE', 'SK_ID_CURR')

In [28]:
df_clean.groupby('WEEKDAY_APPR_PROCESS_START').count().show(30,False)

+--------------------------+-----+
|WEEKDAY_APPR_PROCESS_START|count|
+--------------------------+-----+
|SATURDAY                  |23636|
|THURSDAY                  |34398|
|FRIDAY                    |34134|
|TUESDAY                   |36629|
|WEDNESDAY                 |35241|
|MONDAY                    |34643|
|SUNDAY                    |11532|
+--------------------------+-----+



In [29]:
week_dummy=quick_dummies(df_clean, 'WEEKDAY_APPR_PROCESS_START', 'SK_ID_CURR')

In [30]:
df_clean.groupby('ORGANIZATION_TYPE').count().show(100, False)

+----------------------+-----+
|ORGANIZATION_TYPE     |count|
+----------------------+-----+
|Services              |1448 |
|Trade: type 5         |46   |
|Industry: type 13     |56   |
|Telecom               |523  |
|Bank                  |2252 |
|Industry: type 5      |487  |
|Electricity           |831  |
|Transport: type 2     |1911 |
|Housing               |2525 |
|Insurance             |532  |
|Security              |2996 |
|Postal                |2073 |
|School                |7420 |
|Government            |8281 |
|Security Ministries   |1484 |
|Military              |2074 |
|Realtor               |350  |
|Industry: type 10     |94   |
|Self-employed         |34724|
|Culture               |294  |
|Religion              |64   |
|Industry: type 3      |2831 |
|Trade: type 3         |3234 |
|Industry: type 9      |2823 |
|Advertising           |360  |
|Trade: type 2         |1786 |
|Other                 |11453|
|Police                |2075 |
|Industry: type 12     |315  |
|Trade: 

In [31]:
org_dummy=quick_dummies(df_clean, 'ORGANIZATION_TYPE', 'SK_ID_CURR')

In [32]:
org_dummy

DataFrame[SK_ID_CURR: int, Services: int, Trade: type 5: int, Industry: type 13: int, Telecom: int, Bank: int, Industry: type 5: int, Electricity: int, Transport: type 2: int, Housing: int, Insurance: int, Security: int, Postal: int, School: int, Government: int, Security Ministries: int, Military: int, Realtor: int, Industry: type 10: int, Self-employed: int, Culture: int, Religion: int, Industry: type 3: int, Trade: type 3: int, Industry: type 9: int, Advertising: int, Trade: type 2: int, Other: int, Police: int, Industry: type 12: int, Trade: type 7: int, Construction: int, Industry: type 2: int, Trade: type 6: int, Industry: type 7: int, Hotel: int, Industry: type 11: int, Industry: type 8: int, Agriculture: int, Business Entity Type 3: int, Medicine: int, Restaurant: int, Industry: type 6: int, Emergency: int, Industry: type 4: int, Business Entity Type 2: int, Business Entity Type 1: int, Cleaning: int, Transport: type 3: int, Mobile: int, Transport: type 1: int, Trade: type 4: i

In [33]:
df_clean.groupby('FONDKAPREMONT_MODE').count().show()

+--------------------+------+
|  FONDKAPREMONT_MODE| count|
+--------------------+------+
|    reg oper account| 50328|
|                null|143868|
|       not specified|  3959|
|reg oper spec acc...|  8260|
|    org spec account|  3798|
+--------------------+------+



In [34]:
df_clean = df_clean.drop('FONDKAPREMONT_MODE')

In [35]:
df_clean.groupby('HOUSETYPE_MODE').count().show()

+----------------+------+
|  HOUSETYPE_MODE| count|
+----------------+------+
|            null|105512|
|specific housing|  1124|
|  block of flats|102743|
|  terraced house|   834|
+----------------+------+



In [36]:
df_clean = df_clean.drop('HOUSETYPE_MODE')

In [37]:
df_clean.groupby('WALLSMATERIAL_MODE').count().show()

+------------------+------+
|WALLSMATERIAL_MODE| count|
+------------------+------+
|      Stone, brick| 44395|
|              null|106965|
|        Monolithic|  1221|
|             Mixed|  1618|
|            Wooden|  3795|
|            Others|  1130|
|             Panel| 44867|
|             Block|  6222|
+------------------+------+



In [38]:
df_clean = df_clean.drop('WALLSMATERIAL_MODE')

In [39]:
df_clean.groupby('EMERGENCYSTATE_MODE').count().show()

+-------------------+------+
|EMERGENCYSTATE_MODE| count|
+-------------------+------+
|               null| 99709|
|                 No|108857|
|                Yes|  1647|
+-------------------+------+



In [40]:
df_clean = df_clean.drop('EMERGENCYSTATE_MODE')

In [41]:
df_clean.count()

210213

In [42]:
strcolumnList = [item[0] for item in df_clean.dtypes if item[1].startswith('string')]
strcolumnList

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE']

In [43]:
dummy_df = df_clean.withColumn('CODE_GENDER_DUMMY', (fn.col('CODE_GENDER') == "M").cast('int')).\
            withColumn('CONTRACT_DUMMY_REV', (fn.col('NAME_CONTRACT_TYPE') == "Revolving loans").cast('int')).\
            withColumn('CAR_DUMMY', (fn.col('FLAG_OWN_CAR') == 'Y').cast('int')).\
            withColumn('REALTY_DUMMY', (fn.col('FLAG_OWN_REALTY') == 'Y').cast('int')).\
            drop('CODE_GENDER').\
            drop('NAME_CONTRACT_TYPE').\
            drop('FLAG_OWN_CAR').\
            drop('FLAG_OWN_REALTY')

In [44]:
dummy_df.limit(10).toPandas()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CODE_GENDER_DUMMY,CONTRACT_DUMMY_REV,CAR_DUMMY,REALTY_DUMMY
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,1
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
5,100008,0,0,99000.0,490495.5,27517.5,454500.0,"Spouse, partner",State servant,Secondary / secondary special,...,0.0,0.0,0.0,0.0,1.0,1.0,1,0,0,1
6,100009,0,1,171000.0,1560726.0,41301.0,1395000.0,Unaccompanied,Commercial associate,Higher education,...,0.0,0.0,0.0,1.0,1.0,2.0,0,0,1,1
7,100010,0,0,360000.0,1530000.0,42075.0,1530000.0,Unaccompanied,State servant,Higher education,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,1
8,100012,0,0,135000.0,405000.0,20250.0,405000.0,Unaccompanied,Working,Secondary / secondary special,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,1
9,100014,0,1,112500.0,652500.0,21177.0,652500.0,Unaccompanied,Working,Higher education,...,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1


JOIN THE TABLES CREATED ABOVE AND THEN DROP COLUMNS YOU CREATED DUMMIES FOR

In [45]:
#df_list = [suite_dummy,income_dummy,edu_dummy,fam_dummy,house_dummy,occ_dummy,week_dummy,org_dummy]

#for df in df_list:
    
    #final_df = df_clean.join(df, df_clean.SK_ID_CURR == df.SK_ID_CURR,how='left')

In [46]:
df_join_1 = dummy_df.join(suite_dummy, "SK_ID_CURR",how='left')

In [47]:
df_join_1.count()

210213

In [48]:
df_join_1.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AGE: double (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_EMP_PHONE: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_CONT_MOB

In [49]:
df_join_2 = df_join_1.join(income_dummy, "SK_ID_CURR",how='left')

In [50]:
del df_join_1

In [51]:
df_join_3 = df_join_2.join(edu_dummy, "SK_ID_CURR",how='left')

In [52]:
del df_join_2

In [53]:
df_join_4 = df_join_3.join(fam_dummy,  "SK_ID_CURR",how='left')

In [54]:
del df_join_3

In [55]:
df_join_5 = df_join_4.join(house_dummy,  "SK_ID_CURR",how='left')

In [56]:
df_join_6 = df_join_5.join(occ_dummy,  "SK_ID_CURR",how='left')

In [57]:
del df_join_5

In [58]:
df_join_7 = df_join_6.join(week_dummy,  "SK_ID_CURR",how='left')

In [59]:
del df_join_6

In [60]:
df_join_8 = df_join_7.join(org_dummy,  "SK_ID_CURR",how='left')

In [61]:
del df_join_7

In [62]:
df_final = df_join_8.drop('NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE',
                         'OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','NAME_EDUCATION_TYPE')

In [63]:
del df_join_8

In [64]:
df_final.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AGE: double (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_EMP_PHONE: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_CONT_MOBILE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)
 |-- REGION_RATING_CLIENT: integer (nullable = true)
 |-- REGION_RATING_CLIENT_W_

Work the missing values in exit_source columns

In [65]:
df_final.select([c for c in df.columns if c in ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']]).show()
exit_df = df_final.select([c for c in df.columns if c in ['SK_ID_CURR','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']])

+-------------------+-------------------+-------------------+
|       EXT_SOURCE_1|       EXT_SOURCE_2|       EXT_SOURCE_3|
+-------------------+-------------------+-------------------+
|0.08303696739132256| 0.2629485927471776|0.13937578009978951|
| 0.3112673113812225| 0.6222457752555098|               null|
|               null| 0.5559120833904428| 0.7295666907060153|
|               null| 0.6504416904014653|               null|
|               null| 0.3227382869704046|               null|
|               null| 0.3542247319929012| 0.6212263380626669|
| 0.7747614130547695| 0.7239998516953141| 0.4920600938649263|
|               null| 0.7142792864482229| 0.5406544504453575|
|               null| 0.7466436294590924|               null|
| 0.3197601716755032| 0.6518623334244781|  0.363945238612397|
| 0.4648311169632833| 0.7150418188660659| 0.1766525794312139|
|               null| 0.5669066132460429| 0.7700870700124128|
| 0.7219397686622343| 0.6426562048311103|               null|
|0.11563

Since they are numerical, I will try a mean imputation to fill null values, keeping these columns in for the ml_df below.

In [66]:
def fill_with_mean(df, exclude=set()): 
    stats = df.agg(*(
        fn.avg(c).alias(c) for c in df.columns if c not in exclude
    ))
    return df.na.fill(stats.first().asDict())

In [67]:
exit_df_clean = fill_with_mean(exit_df, ['SK_ID_CURR'])

In [68]:
del exit_df

In [69]:
exit_df_clean = exit_df_clean.withColumnRenamed("EXT_SOURCE_1", "EXT_SOURCE_1_clean").\
                withColumnRenamed("EXT_SOURCE_2", "EXT_SOURCE_2_clean").\
                withColumnRenamed("EXT_SOURCE_3", "EXT_SOURCE_3_clean")

In [70]:
exit_df_clean.show(10)

+----------+-------------------+------------------+-------------------+
|SK_ID_CURR| EXT_SOURCE_1_clean|EXT_SOURCE_2_clean| EXT_SOURCE_3_clean|
+----------+-------------------+------------------+-------------------+
|    100002|0.08303696739132256|0.2629485927471776|0.13937578009978951|
|    100003| 0.3112673113812225|0.6222457752555098| 0.5009057167336302|
|    100004| 0.4796136737107814|0.5559120833904428| 0.7295666907060153|
|    100006| 0.4796136737107814|0.6504416904014653| 0.5009057167336302|
|    100007| 0.4796136737107814|0.3227382869704046| 0.5009057167336302|
|    100008| 0.4796136737107814|0.3542247319929012| 0.6212263380626669|
|    100009| 0.7747614130547695|0.7239998516953141| 0.4920600938649263|
|    100010| 0.4796136737107814|0.7142792864482229| 0.5406544504453575|
|    100012| 0.4796136737107814|0.7466436294590924| 0.5009057167336302|
|    100014| 0.3197601716755032|0.6518623334244781|  0.363945238612397|
+----------+-------------------+------------------+-------------

Read in credit card balance data

In [71]:
card_balance = spark.read.format("csv").\
    option("header", "true").\
    option("inferSchema", "true").\
     load("credit_card_balance.csv")

In [72]:
card_balance.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- MONTHS_BALANCE: integer (nullable = true)
 |-- AMT_BALANCE: double (nullable = true)
 |-- AMT_CREDIT_LIMIT_ACTUAL: integer (nullable = true)
 |-- AMT_DRAWINGS_ATM_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_OTHER_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_POS_CURRENT: double (nullable = true)
 |-- AMT_INST_MIN_REGULARITY: double (nullable = true)
 |-- AMT_PAYMENT_CURRENT: double (nullable = true)
 |-- AMT_PAYMENT_TOTAL_CURRENT: double (nullable = true)
 |-- AMT_RECEIVABLE_PRINCIPAL: double (nullable = true)
 |-- AMT_RECIVABLE: double (nullable = true)
 |-- AMT_TOTAL_RECEIVABLE: double (nullable = true)
 |-- CNT_DRAWINGS_ATM_CURRENT: double (nullable = true)
 |-- CNT_DRAWINGS_CURRENT: integer (nullable = true)
 |-- CNT_DRAWINGS_OTHER_CURRENT: double (nullable = true)
 |-- CNT_DRAWINGS_POS_CURRENT: double (nullable = true)
 |

In [73]:
card_balance.limit(10).toPandas()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.500,0.0,877.500,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,0.0,0.000,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.000,0.0,0.000,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.000,0.0,0.000,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
5,2646502,380010,-7,82903.815,270000,0.0,0.000,0.0,0.000,4449.105,...,82773.315,82773.315,0.0,0,0.0,0.0,2.0,Active,7,0
6,1079071,171320,-6,353451.645,585000,67500.0,67500.000,0.0,0.000,14684.175,...,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0
7,2095912,118650,-7,47962.125,45000,45000.0,45000.000,0.0,0.000,0.000,...,47962.125,47962.125,1.0,1,0.0,0.0,51.0,Active,0,0
8,2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.500,...,286831.575,286831.575,3.0,8,0.0,5.0,3.0,Active,0,0
9,1235299,203885,-5,201261.195,225000,76500.0,111026.700,0.0,34526.700,6338.340,...,197224.695,197224.695,3.0,9,0.0,6.0,38.0,Active,0,0


In [74]:
id_count = card_balance.groupby('SK_ID_CURR').count()

In [75]:
sum_limit = card_balance.groupby('SK_ID_CURR').agg(F.sum('AMT_CREDIT_LIMIT_ACTUAL'))

In [76]:
avg_limit = id_count.join(sum_limit, 'SK_ID_CURR')

In [77]:
avg_limit = avg_limit.withColumn('avg_cred_lim', (F.col('sum(AMT_CREDIT_LIMIT_ACTUAL)')/(F.col('count'))))
avg_limit.show(10)

+----------+-----+----------------------------+------------------+
|SK_ID_CURR|count|sum(AMT_CREDIT_LIMIT_ACTUAL)|      avg_cred_lim|
+----------+-----+----------------------------+------------------+
|    100170|    7|                     2362500|          337500.0|
|    100274|   25|                     7605000|          304200.0|
|    100800|   11|                      990000|           90000.0|
|    102798|   22|                     9900000|          450000.0|
|    103747|    6|                           0|               0.0|
|    104064|   28|                     5850000|208928.57142857142|
|    105536|   92|                    13680000|148695.65217391305|
|    105665|   96|                     6480000|           67500.0|
|    105796|   96|                     8010000|           83437.5|
|    106544|   10|                      450000|           45000.0|
+----------+-----+----------------------------+------------------+
only showing top 10 rows



In [78]:
avg_limit = avg_limit.drop('sum(AMT_CREDIT_LIMIT_ACTUAL)','count')

In [79]:
amt_payment_toal = card_balance.groupby('SK_ID_CURR').agg(F.sum('AMT_PAYMENT_TOTAL_CURRENT'))
amt_payment_toal.show(10)

+----------+------------------------------+
|SK_ID_CURR|sum(AMT_PAYMENT_TOTAL_CURRENT)|
+----------+------------------------------+
|    294744|                     171046.53|
|    348429|          1.4755217219999999E7|
|    188986|            2263453.6050000004|
|    393796|                           0.0|
|    119813|                    172632.195|
|    156527|                           0.0|
|    222048|                    635906.835|
|    387852|            233971.28999999998|
|    115528|                           0.0|
|    357089|            41704.380000000005|
+----------+------------------------------+
only showing top 10 rows



In [80]:
amt_rec_toal = card_balance.groupby('SK_ID_CURR').agg(F.sum('AMT_TOTAL_RECEIVABLE'))

In [81]:
id_sums = card_balance.groupby('SK_ID_CURR').agg(F.sum('AMT_BALANCE'))

In [82]:
joined_cards = id_sums.join(id_count, 'SK_ID_CURR')

In [83]:
joined_cards = joined_cards.join(amt_rec_toal, 'SK_ID_CURR')

In [84]:
joined_cards = joined_cards.join(amt_payment_toal, 'SK_ID_CURR')

In [85]:
joined_cards = joined_cards.join(avg_limit, 'SK_ID_CURR')

In [86]:
joined_cards.show(10)

+----------+-----------------+-----+-------------------------+------------------------------+------------------+
|SK_ID_CURR| sum(AMT_BALANCE)|count|sum(AMT_TOTAL_RECEIVABLE)|sum(AMT_PAYMENT_TOTAL_CURRENT)|      avg_cred_lim|
+----------+-----------------+-----+-------------------------+------------------------------+------------------+
|    100170|      2175422.175|    7|               2138734.71|                       87750.0|          337500.0|
|    100274|              0.0|   25|                      0.0|                           0.0|          304200.0|
|    100800|         13984.02|   11|                 12638.52|                       11250.0|           90000.0|
|    102798|7886136.824999999|   22|        7896447.090000001|                      407038.5|          450000.0|
|    103747|              0.0|    6|                      0.0|                     1125000.0|               0.0|
|    104064|              0.0|   28|                      0.0|                           0.0|208

In [87]:
del card_balance 

In [88]:
joined_df = joined_cards.withColumn('avg_balance', (F.col('sum(AMT_BALANCE)')/(F.col('count'))))

In [89]:
del joined_cards

In [90]:
joined_df.show(10)

+----------+-----------------+-----+-------------------------+------------------------------+------------------+------------------+
|SK_ID_CURR| sum(AMT_BALANCE)|count|sum(AMT_TOTAL_RECEIVABLE)|sum(AMT_PAYMENT_TOTAL_CURRENT)|      avg_cred_lim|       avg_balance|
+----------+-----------------+-----+-------------------------+------------------------------+------------------+------------------+
|    100170|      2175422.175|    7|               2138734.71|                       87750.0|          337500.0| 310774.5964285714|
|    100274|              0.0|   25|                      0.0|                           0.0|          304200.0|               0.0|
|    100800|         13984.02|   11|                 12638.52|                       11250.0|           90000.0|1271.2745454545454|
|    102798|7886136.824999999|   22|        7896447.090000001|                      407038.5|          450000.0| 358460.7647727272|
|    103747|              0.0|    6|                      0.0|              

In [91]:
joined_df = joined_df.drop('sum(AMT_BALANCE)','count')
joined_df.show(10)

+----------+-------------------------+------------------------------+------------------+------------------+
|SK_ID_CURR|sum(AMT_TOTAL_RECEIVABLE)|sum(AMT_PAYMENT_TOTAL_CURRENT)|      avg_cred_lim|       avg_balance|
+----------+-------------------------+------------------------------+------------------+------------------+
|    100170|               2138734.71|                       87750.0|          337500.0| 310774.5964285714|
|    100274|                      0.0|                           0.0|          304200.0|               0.0|
|    100800|                 12638.52|                       11250.0|           90000.0|1271.2745454545454|
|    102798|        7896447.090000001|                      407038.5|          450000.0| 358460.7647727272|
|    103747|                      0.0|                     1125000.0|               0.0|               0.0|
|    104064|                      0.0|                           0.0|208928.57142857142|               0.0|
|    105536|               8

In [92]:
joined_df = joined_df.withColumnRenamed("sum(AMT_TOTAL_RECEIVABLE)", "tot_amt_rec").\
            withColumnRenamed("sum(AMT_PAYMENT_TOTAL_CURRENT)", "tot_pay_curr")

In [93]:
joined_df.show(10)

+----------+-----------------+------------+------------------+------------------+
|SK_ID_CURR|      tot_amt_rec|tot_pay_curr|      avg_cred_lim|       avg_balance|
+----------+-----------------+------------+------------------+------------------+
|    100170|       2138734.71|     87750.0|          337500.0| 310774.5964285714|
|    100274|              0.0|         0.0|          304200.0|               0.0|
|    100800|         12638.52|     11250.0|           90000.0|1271.2745454545454|
|    102798|7896447.090000001|    407038.5|          450000.0| 358460.7647727272|
|    103747|              0.0|   1125000.0|               0.0|               0.0|
|    104064|              0.0|         0.0|208928.57142857142|               0.0|
|    105536|       8784518.49|    738135.0|148695.65217391305| 95493.16858695653|
|    105665|       3289736.97|   377983.35|           67500.0|       34276.97625|
|    105796|4047749.819999999|    304200.0|           83437.5| 42141.79687499999|
|    106544|    

In [94]:
df_final_1 = df_final.join(joined_df, 'SK_ID_CURR', how='left')

In [95]:
del df_final

In [96]:
df_final_2 = df_final_1.join(exit_df_clean, 'SK_ID_CURR', how='left')

In [97]:
del df_final_1

In [98]:
df_final_2 = df_final_2.drop('EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3')

Need to fill the features created with mean in the joined dataframe so they are not dropped.

In [99]:
from pyspark.sql.functions import mean

avg_avg_balance = df_final_2.agg(mean(df_final_2['avg_balance'])).collect()[0]
avg_avg_cred_lim = df_final_2.agg(mean(df_final_2['avg_cred_lim'])).collect()[0]
avg_tot_amt_rec = df_final_2.agg(mean(df_final_2['tot_amt_rec'])).collect()[0]
avg_tot_pay_curr = df_final_2.agg(mean(df_final_2['tot_pay_curr'])).collect()[0]

In [100]:
avg_avg_balance

Row(avg(avg_balance)=74491.21812434169)

In [101]:
avg_avg_balance_num = 74491.21812434169

In [102]:
avg_avg_cred_lim

Row(avg(avg_cred_lim)=214360.63570436096)

In [103]:
avg_avg_cred_lim_num = 214360.63570436096

In [104]:
avg_tot_pay_curr

Row(avg(tot_pay_curr)=286723.22028139123)

In [105]:
avg_tot_pay_curr_num = 286723.22028139123

In [106]:
avg_tot_amt_rec_num = 2198089.5999815087

In [107]:
df_final_2 = df_final_2.fillna(avg_avg_balance_num, subset=['avg_balance'])
df_final_2 = df_final_2.fillna(avg_avg_cred_lim_num, subset=['avg_cred_lim'])
df_final_2 = df_final_2.fillna(avg_tot_amt_rec_num, subset=['tot_amt_rec'])
df_final_2 = df_final_2.fillna(avg_tot_pay_curr_num, subset=['tot_pay_curr'])

In [108]:
# import pandas as pd
# df_pandas = df_final_2.toPandas()

In [109]:
# def missing_values_table(df):
        # Total missing values
        # mis_val = df.isnull().sum()
        
        # Percentage of missing values
        # mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        # mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        # mis_val_table_ren_columns = mis_val_table.rename(
        # columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        # mis_val_table_ren_columns = mis_val_table_ren_columns[
            # mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        # '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        # print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            # "There are " + str(mis_val_table_ren_columns.shape[0]) +
              # " columns that have missing values.")
        
        # Return the dataframe with missing information
        # return mis_val_table_ren_columns

In [110]:
# missing_values_table(df_pandas)

In [111]:
# get correlations
# correlations = df_pandas.corr()['TARGET'].sort_values()

# Display correlations
# print('Most Positive Correlations:\n', correlations.tail(15))
# print('\nMost Negative Correlations:\n', correlations.head(15))

build model_df

In [112]:
ml_df = df_final_2.drop('SK_ID_CURR')

In [113]:
def drop_null_columns(df):
    """
    This function drops all columns which contain null values.
    :param df: A PySpark DataFrame
    """
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > 0]
    df = df.drop(*to_drop)
    return df

In [114]:
ml_df_1 = drop_null_columns(ml_df)

Write to .csv:

In [117]:
ml_df_1.coalesce(1).write.format('com.databricks.spark.csv').save('ml_df_test.csv',header = 'true')

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 55740)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError
----------------------------------------
